# Sistema Jurídico RAG Avançado (Notebook Demo)

Este notebook apresenta uma implementação de referência baseada na arquitetura de 4 camadas descrita na documentação técnica. Ele foca em:
- Extração estruturada (Docling)
- Indexação baseada em raciocínio (PageIndex)
- Gestão de contexto conversacional (ChatIndex)
- Persistência auditável (Google Drive)

Inclui também um chat jurídico robusto com rastreabilidade e logs de auditoria.


## Dependências e configuração

Este notebook foi estruturado para rodar localmente ou em ambientes como Colab. Ajuste os caminhos e chaves conforme necessário.

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional
import hashlib
import json

# Configuração base (ajuste conforme seu ambiente)
@dataclass(frozen=True)
class ConfigSistema:
    pageindex_api_url: str = "https://api.pageindex.ai/v1/index"
    pageindex_api_key: str = "SUA_CHAVE_AQUI"
    chatindex_dir: Path = Path("./chatindex")
    drive_root: Path = Path("./Juridico_Unificado")
    audit_dir: Path = Path("./Juridico_Unificado/05_Auditoria")
    cache_dir: Path = Path("./cache_juridico")

config = ConfigSistema()
config

## Camada 1: Orquestração

A classe `SistemaJuridicoUnificado` atua como facade, coordenando extração, chunking, indexação, scraping e auditoria.

In [ ]:
class SistemaAuditoriaUnificado:
    def __init__(self, audit_dir: Path):
        self.audit_dir = audit_dir
        self.audit_dir.mkdir(parents=True, exist_ok=True)
        self.log_central: List[Dict[str, Any]] = []
        self.hash_registry: Dict[str, Dict[str, str]] = {}

    def registrar_evento(self, categoria: str, evento: Dict[str, Any]) -> str:
        evento_id = f"evt_{hashlib.md5(str(evento).encode()).hexdigest()[:10]}"
        evento["timestamp"] = evento.get("timestamp", datetime.now().isoformat())

        if self.log_central:
            evento["hash_anterior"] = self.hash_registry[self.log_central[-1]["evento_id"]]["hash"]

        hash_atual = hashlib.md5(json.dumps(evento, sort_keys=True).encode()).hexdigest()
        self.hash_registry[evento_id] = {"hash": hash_atual, "timestamp": evento["timestamp"]}

        registro = {**evento, "evento_id": evento_id, "hash_atual": hash_atual}
        self.log_central.append(registro)
        self._persistir_log(categoria, registro)
        return evento_id

    def _persistir_log(self, categoria: str, registro: Dict[str, Any]) -> None:
        destino = self.audit_dir / f"{categoria}.jsonl"
        with destino.open("a", encoding="utf-8") as handle:
            handle.write(json.dumps(registro, ensure_ascii=False) + "\n")


class SistemaJuridicoUnificado:
    def __init__(self, config: ConfigSistema):
        self.config = config
        self.auditoria = SistemaAuditoriaUnificado(config.audit_dir)

    def processar_documento(self, documento_path: Path) -> Dict[str, Any]:
        self.auditoria.registrar_evento("processamento", {
            "tipo": "inicio_processamento",
            "documento": documento_path.name,
        })
        # Placeholder para fluxo completo (Docling -> Chunking -> PageIndex)
        extracao = {"documento_id": documento_path.stem, "texto_completo": documento_path.read_text()}
        chunks = [extracao["texto_completo"]]
        arvore = {"documento_id": documento_path.stem, "estrutura_arvore": {"raiz": {"titulo": documento_path.stem}}}

        self.auditoria.registrar_evento("processamento", {
            "tipo": "fim_processamento",
            "documento": documento_path.name,
            "chunks_gerados": len(chunks),
        })
        return {"extracao": extracao, "chunks": chunks, "arvore": arvore}

    def responder_consulta(self, consulta: str, contexto: Dict[str, Any]) -> Dict[str, Any]:
        self.auditoria.registrar_evento("consultas", {
            "tipo": "consulta",
            "consulta": consulta,
        })
        resposta = {
            "consulta": consulta,
            "resposta": f"Resumo jurídico para: {consulta}",
            "fontes": contexto.get("fontes", []),
        }
        self.auditoria.registrar_evento("consultas", {
            "tipo": "resposta",
            "consulta": consulta,
            "hash_resposta": hashlib.md5(json.dumps(resposta, ensure_ascii=False).encode()).hexdigest(),
        })
        return resposta


## Chat Jurídico Robusto

Este fluxo simula um chat jurídico que:
- Mantém histórico de contexto
- Registra auditoria de cada interação
- Retorna respostas com fontes rastreáveis


In [ ]:
class ChatJuridico:
    def __init__(self, sistema: SistemaJuridicoUnificado):
        self.sistema = sistema
        self.historico: List[Dict[str, Any]] = []

    def enviar(self, consulta: str, fontes: Optional[List[str]] = None) -> Dict[str, Any]:
        contexto = {"fontes": fontes or []}
        resposta = self.sistema.responder_consulta(consulta, contexto)
        self.historico.append({"consulta": consulta, "resposta": resposta})
        return resposta


sistema = SistemaJuridicoUnificado(config)
chat = ChatJuridico(sistema)

chat.enviar("Quais são os requisitos para tutela de urgência?", ["STJ", "Planalto"])

## Próximos passos

1. Substituir os placeholders por integrações reais (Docling, PageIndex API).
2. Implementar scraping com Playwright e rate limiting.
3. Enriquecer o ChatIndex com histórico persistente.
4. Adicionar testes de regressão e avaliação de qualidade.
